Problem: mamy szyfr Hilla, działającym poprzez mnożenie macierzy w pierścieniu modulo 26. Na podstawie wiadomości jawnej $m$ oraz kryptogramu $e$ chcemy znaleźć wartości macierzy Hilla $A$ oraz odpowiadającą jej macierz odwrotną $A^{-1}$.

Przykład: 

- wiadomość jawna $m = pies = m_1m_2 = (pi)(es) = [15, 8][4, 18]$
- kryptogram $e = ntqi = e_1e_2 = (nt)(qi) = [13, 19][16, 8]$
- macierz Hilla:

$A = \begin{bmatrix}
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{bmatrix} = \begin{bmatrix}
7 & 8\\
11 & 11
\end{bmatrix}$

Operacja szyfrowania:

$
A m = e\\
A m_1 = e_1 = \begin{bmatrix} 7 & 8\\ 11 & 11 \end{bmatrix} \begin{bmatrix} 15 \\ 8 \end{bmatrix} \mod 26 = \begin{bmatrix} 13 \\ 19 \end{bmatrix} = nt \\
A m_2 = e_2 = \begin{bmatrix} 7 & 8\\ 11 & 11 \end{bmatrix} \begin{bmatrix} 4 \\ 18 \end{bmatrix} \mod 26 = \begin{bmatrix} 16 \\ 8 \end{bmatrix} = qi 
$

Sprawdźmy teraz, czy obliczenia się zgadzają:

In [2]:
from numpy import array

A = array([[7,8],[11,11]])
m1 = array([15,8])
m2 = array([4,18])

e1 = A.dot(m1) % 26
e2 = A.dot(m2) % 26
print(f"Szyfrogram e1 = {e1}")
print(f"Szyfrogram e2 = {e2}")

Szyfrogram e1 = [13 19]
Szyfrogram e2 = [16  8]


Jeśli chcemy wyznaczyć macierz odwrotną $B = A^{-1}$, to stosujemy wzór:

$B = A^{-1} = \frac{A^D}{\det A},$

gdzie $A^D$ jest macierzą dołączoną - transponowaną macierzą dopełnień algebraicznych, gdzie dopełnienie algebraiczne dla elementu $a_{ij}$ to wyznacznik minora tej macierzy, tj. macierzy powstałej przez usunięcie $i$-tego wiersza oraz $j$-tej kolumny, przemnożonego przez wartość $(-1)^{i+j}$. W przypadku macierzy 2x2 wyznaczanie minora jest trywialne, bo będzie to po prostu wartość elementu przeciwległego do elementu, którego dopełnienie chcemy wyznaczyć.

Zatem dla macierzy $A = \begin{bmatrix}7 & 8\\11&11\end{bmatrix}$ macierz dołączona $A^D = \begin{bmatrix} 11 & -8 \\ -11 & 7 \end{bmatrix} \mod 26 = \begin{bmatrix} 11 & 18 \\ 15 & 19\end{bmatrix}$.

Poniżej sprawdzimy, czy rzeczywiście $A^D$ jest macierzą dołączoną. Spodziewamy się, że po przemnożeniu przez $A$ otrzymamy wielokrotność macierzy jednostkowej, tj. macierz z tą samą wartością tylko na diagonali:

In [3]:
from numpy import array
A = array([[7,8],[11,11]])
AD = array([[11,18],[15,7]])

A.dot(AD) % 26 # spodziewamy się wielokrotności macierzy [[[1 0] [0 1]]

array([[15,  0],
       [ 0, 15]], dtype=int32)


Wyznacznik macierzy $A$ to $\det A = 7 \cdot 11 - 8 \cdot 11 = 77 - 88 = -11 \mod 26 = 15 \mod 26 = 15$. Odwrotność $\det A$ w pierścieniu modulo 26 to $15^{-1} = 7 \mod 26 = 7$. 

(bo $15 \cdot 7 \mod 26 = 105 \mod 26 = 1$)

Macierz $A^{-1}$ to $A^D / \det A = A^D \cdot (\det A)^{-1} \mod 26 = \begin{bmatrix} 11 & 18 \\ 15 & 7\end{bmatrix} \cdot 7 \mod 26 = \begin{bmatrix} 77 & 126 \\ 105 & 49\end{bmatrix} \mod 26 = \begin{bmatrix} 25 & 22 \\ 1 & 23\end{bmatrix}$

Sprawdzimy teraz, czy faktycznie obliczenia się zgadzają:

In [4]:
from numpy import array
A = array([[7,8],[11,11]])
B = array([[25,22],[1,23]])

A.dot(B) % 26 # spodziewamy się macierzy [[[1 0] [0 1]]

array([[1, 0],
       [0, 1]], dtype=int32)

Istotnie, macierz $B$ jest odwrotnością macierzy $A$ (mod 26). Sprawdzamy, czy macierz $B$ poprawnie deszyfruje kryptogram $e$:

In [5]:
from numpy import array

B = array([[25,22],[1,23]])
e1 = array([13,19])
e2 = array([16,8])

m1 = B.dot(e1) % 26
m2 = B.dot(e2) % 26
print(f"Wiadomość jawna m1 = {m1}")
print(f"Wiadomość jawna m2 = {m2}")

Wiadomość jawna m1 = [15  8]
Wiadomość jawna m2 = [ 4 18]


Deszyfrowanie działa poprawnie.

Chcielibyśmy teraz stworzyć algorytm wyznaczający macierze $A$ $B$ na podstawie wiadomości jawnej $m$ oraz kryptogramu $e$.

Zauważmy, że:

$A m = e \mod 26,$

tj.

$\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} \begin{bmatrix} m_{11} \\ m_{12}\end{bmatrix} = \begin{bmatrix} e_{11}\\e_{12}\end{bmatrix},$

oraz:

$\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} \begin{bmatrix} m_{21} \\ m_{22}\end{bmatrix} = \begin{bmatrix} e_{21}\\e_{22}\end{bmatrix}.$

Zestawiając razem:

$\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} \begin{bmatrix} m_{11} & m_{21} \\ m_{12} & m_{22}\end{bmatrix} = \begin{bmatrix} e_{11} & e_{21}\\e_{12} & e_{22}\end{bmatrix},$

czyli:

$ A M = E.$

Jeśli mamy $M$ oraz $E$, to wzór na $A$ to (zakładajac że $M$ oraz $E$ mają niezerowe wyznaczniki):

$ A = E M^{-1}.$

Musimy zatem wyznaczyć $M^{-1}$. Skorzystamy z tej samej metody co poprzednio:

$M^{-1} = \frac{M^D}{\det M}.$

$ M = \begin{bmatrix} m_{11} & m_{21} \\ m_{12} & m_{22}\end{bmatrix} = \begin{bmatrix}15 & 4\\8 & 18\end{bmatrix}$

$ M^D = \begin{bmatrix} m_{22} & -m_{21} \\ -m_{12} & m_{11}\end{bmatrix} = \begin{bmatrix} 18 & -4 \\ -8 & 15\end{bmatrix} \mod 26 = \begin{bmatrix} 18 & 22 \\ 18 & 15\end{bmatrix}$

Sprawdzamy nasze obliczenia. $M\cdot M^D$ powinno dawać wielokrotność macierzy jednostkowej modulo 26:

In [12]:
from numpy import array

M = array([[15,4],[8,18]])
MD = array([[18,22],[18,15]])

M.dot(MD) % 26 # spodziewamy się wielokrotności macierzy [[[1 0] [0 1]]

array([[4, 0],
       [0, 4]], dtype=int32)

Wyznacznik macierzy M:

$\det M = m_{11} \cdot m_{22} - m_{12} \cdot m_{21} = 15 \cdot 18 - 8 \cdot 4 = 270 - 32 = 238 \mod 26 = 4.$

Tutaj mamy problem - 4 nie ma odwrotności modulo 26. Powinniśmy znaleźć inną wiadomość 4 literową, która da nam wyznacznik macierzy $M$ wględnie pierwszy z 26. Wtedy będziemy mogli wyznaczyć macierz $M^{-1}$.

In [16]:
from numpy import array
from numpy.linalg import det

M = array([[1,4],[3,5]])

print(det(M)) # liczymy wyznacznik macierzy M, czyli wiadomości jawnej

-7.000000000000001


Widzimy, że wiadomość `bdef` będzie odpowiednia, bo jej wyznacznik wynosi -7, który jest względnie pierwszy z 26. Wiemy również, że szyfrowanie wiadomości `bdef` da nam kryptogram `ntqi`:

In [14]:
from numpy import array
A = array([[7,8],[11,11]])
M = array([[1,4],[3,5]])

A.dot(M) % 26 # liczymy kryptogram

array([[ 5, 16],
       [18, 21]], dtype=int32)

In [15]:
to_msg = lambda x: ''.join([chr(i+ord('a')) for i in x])

print(to_msg([5,18,16,21])) # sprawdzamy jak wygląda tekst kryptogramu

fsqv


Zatem:

$M = \begin{bmatrix} 1 & 4\\3 & 5\end{bmatrix}$

$E = \begin{bmatrix} 5 & 16\\18&21\end{bmatrix}$

$A = E M^{-1}$

$M^{-1} = \frac{M^D}{\det M}$

$M^D = \begin{bmatrix} 5 & -4\\-3 & 1\end{bmatrix} \mod 26 = \begin{bmatrix} 5 & 22\\23 & 1\end{bmatrix}$

$\det M = 1 \cdot 5 - 3 \cdot 4 = 5 - 12 = -7 \mod 26 = 19$

$(\det M)^{-1} = 19^{-1} \mod 26= 11$

$M^{-1} = M^D \cdot (\det M)^{-1} = \begin{bmatrix} 5 & 22\\23 & 1\end{bmatrix} \cdot 11 \mod 26 = \begin{bmatrix} 55 & 242\\253 & 11\end{bmatrix} \mod 26 = \begin{bmatrix} 3 & 8\\19 & 11\end{bmatrix}$

Sprawdzamy, czy nasze obliczenia są poprawne:

In [10]:
from numpy import array

M = array([[1,4],[3,5]])
MI = array([[3,8],[19,11]]) # odwrotność M

M.dot(MI) % 26 # spodziewamy się macierzy [[[1 0] [0 1]]

array([[1, 0],
       [0, 1]], dtype=int32)

Ostatecznie, obliczamy wynik $E M^{-1}$:

In [13]:
from numpy import array

MI = array([[3,8],[19,11]]) # odwrotność M
E = array([[5,16],[18,21]])

E.dot(MI) % 26 # spodziewamy się macierzy A, tj. [[7 8] [11 11]]

array([[ 7,  8],
       [11, 11]], dtype=int32)

Zatem nasza metoda pozwoliła wstecznie wyznaczyć

$A = \begin{bmatrix} 7 & 8\\11&11\end{bmatrix}$

co zgadza się z wynikiem z poprzedniego przykładu. Macierz $A^{-1}$ możemy wyznaczyć na podstawie macierzy $A$, albo korzystając z przekształcenia:

$A^{-1} = (E M^{-1})^{-1} = M E^{-1}$